In [ ]:
from ecell4 import *
from ecell4.extra.ensemble import ensemble_simulations
from numpy import pi, array


duration = 1000.0

def singlerun(radius, intrinsic=False, n=1, solver='ode'):
    print('radius = {}, intrinsic = {}, n= {}, solver = {}'.format(radius, intrinsic, n, solver))
    L = cbrt(10.0) # 1.0
    N = 12
    D = 1.0
    V = L ** 3

    if solver == 'egfrd':
        # M = max(int(min(cbrt(N), L / (19 * 2 * radius))), 3)
        M = max(int(min(cbrt(N), L / (2 * radius))), 3)
        print("M = {}".format(M))
        solver = ('egfrd', Integer3(M, M, M))
    elif solver == 'spatiocyte':
        solver = ('spatiocyte', radius)
        V = spatiocyte.SpatiocyteWorld.calculate_volume(ones() * L, radius)

    v = 4.0 / 3.0 * pi * (radius ** 3)
    print('phi = {}'.format((N + 1) * v / V))

    k0 = 0.1 / V
    print('k0 = {}'.format(k0))
    keff = k0 * V / N * (V / 1)
    # kD = 4 * pi * (radius * 20) * (D * 2)
    kD = 4 * pi * (radius * 2) * (D * 2)
    k1 = keff * kD / (kD - keff)
    print('keff = {}, kD = {}'.format(keff, kD))
    if keff >= kD:
        raise ValueError('An effective rate cannot be faster than the collision rate.')
    print('k1 = {}, ka_factor = {}'.format(k1, k1 / kD))

    with species_attributes():
        A | {'D': D, 'radius': radius}
        B | {'D': D, 'radius': radius}

    with reaction_rules():
        ~A > A | k0
        # A > ~A | 1.0 / N
        A + B > B | (k1 if intrinsic else keff)

    m = get_model()

    if n > 1:
        obs = ensemble_simulations(duration, model=m, y0={'B': 1}, return_type='observer', species_list=['A'], solver=solver, volume=ones() * L, n=n, nproc=30, method='multiprocessing')
    else:
        obs = run_simulation(duration, model=m, y0={'B': 1}, return_type='observer', species_list=['A'], solver=solver, volume=ones() * L, )
    return obs

radius = 0.005
ntrials = 55
obs1 = singlerun(radius, True, ntrials, 'spatiocyte')
obs2 = singlerun(radius * 10, True, ntrials, 'spatiocyte')
obs3 = singlerun(radius, True, ntrials, 'egfrd')
obs4 = singlerun(radius * 10, True, ntrials, 'egfrd')
obs5 = singlerun(radius, False, 1, 'ode')

# viz.plot_number_observer_with_matplotlib(obs1, '-', obs2, '-',  obs3, '--', filename="birthdeath.png")

import matplotlib.pylab as plt

data1 = array(obs1.data()).T
err1 = array(obs1.error()).T
plt.errorbar(data1[0], data1[1], yerr=err1[1], fmt='r-', label='Spatiocyte (r=0.005)')
data2 = array(obs2.data()).T
err2 = array(obs2.error()).T
plt.errorbar(data2[0], data2[1], yerr=err2[1], fmt='g-', label='Spatiocyte (r=0.05)')
data3 = array(obs3.data()).T
err3 = array(obs3.error()).T
plt.errorbar(data3[0], data3[1], yerr=err3[1], fmt='b-', label='eGFRD (r=0.005)')
data4 = array(obs4.data()).T
err4 = array(obs4.error()).T
plt.errorbar(data4[0], data4[1], yerr=err4[1], fmt='c-', label='eGFRD (r=0.05)')
data5 = array(obs5.data()).T
plt.plot(data5[0], data5[1], 'k--', label='ode')
plt.legend(loc='best', shadow=True)
plt.xlabel('Time')
plt.ylabel('The Number of Molecules')
plt.xlim(0, duration)
plt.savefig("birthdeath.png")
# plt.show()
